Calculate the Patterson Function from the Structure Factors calculated by X-ray Utils. Implementing equation 14.15 in the VESTA manual (http://jp-minerals.org/vesta/archives/VESTA_Manual.pdf)

In [ ]:
### import modules including x-ray utilities
import numpy as np
import os
import xrayutilities as xu
from xrayutilities.materials.cif import CIFFile
from xrayutilities.materials.material import Crystal

In [ ]:
## load the CIF, in this case of the bscco single crystal
xu_cif_bscco = CIFFile("bscco.cif")
xu_cif_bscco = Crystal(name=r"BSCCO", lat=xu_cif_bscco.SGLattice())

XU.materials: Wyckoff positions missing, using P1


In [ ]:
### lattice parameters and x-ray energy
a = 5.40540
b = 5.40160
c = 30.71520
energy= 300090

jj = complex(0, 1)

In [ ]:
### set up reciprocal space
h_max = 7
k_max = 7
l_max = 40

res_x = a  /h_max
res_y = b / k_max
res_z = c / l_max

N_x = 2*int(a/res_x)
N_y = 2*int(b/res_y) 
N_z = 2*int(c/res_z)

print(N_x, N_y, N_z)
print(res_x, res_y, res_z)

x_pos = np.linspace(0, a, num=N_x, endpoint=True)
y_pos = np.linspace(0, b, num=N_y, endpoint=True)
z_pos = np.linspace(0, c, num=N_z, endpoint=True)



In [ ]:
### setup to calculate the structure factor for each reflection
u = np.linspace(-h_max, 1+h_max, num=N_x+1, endpoint=False)
v = np.linspace(-k_max, 1+k_max, num=N_y+1, endpoint=False)
w = np.linspace(-l_max, 1+l_max, num=N_z+1, endpoint=False)

print(u, v, w)

## patterson function to be filled, called e_rho as in equation 14.15
e_rho = np.zeros(((N_x, N_y, N_z)), dtype=complex)
array_to_sum = np.zeros(((len(u), len(v), len(w))), dtype=complex)

### real space grid for 3D PDF grid
x=np.linspace(0, 1, N_x)
y=np.linspace(0, 1, N_y)
z=np.linspace(0, 1, N_z)

In [ ]:
### calculate the structure factor for each reflection using x-ray utils 
strucfacts = np.zeros((h_max+1,k_max+1,l_max+1))
for tt in range(h_max+1):
    for uu in range(k_max+1):
        for vv in range(l_max+1):
            qvec = xu_cif_bscco.Q([tt,uu,vv])
            F = np.abs(xu_cif_bscco.StructureFactor(qvec, energy))
            strucfacts[tt][uu][vv] = np.abs(F)**2
            
            

In [ ]:
### loop for calculating the patterson at each point in on the real space grid
counter = 0
for dd in range(len(x)):
    for ee in range(len(y)):
        for ff in range(len(z)):
            if((counter%100)==0):
                print(str(counter)+' ', end=' ')
            
            for d in range(0, len(u), 1):
                for e in range(0, len(v), 1):
                    for f in range(0, len(w), 1):
                        array_to_sum[d][e][f] = (strucfacts[int(np.abs(u[d]))][int(np.abs(v[e]))][int(np.abs(w[f]))])*np.exp(-2*np.pi*jj*(u[d]*x[dd]+v[e]*y[ee]+w[f]*z[ff]))    
            e_rho[dd][ee][ff] = np.sum(array_to_sum)
            counter = counter+1
    
print(e_rho.shape)

In [ ]:
### save the 3D PDF
with open('e_rho_xrayutil.npy', 'wb') as f:
    np.save(f, e_rho)

In [ ]:
### plot the 3D PDF
import matplotlib.pyplot as plt
%matplotlib notebook
e_rho_real = e_rho.real
print(e_rho.real.shape)
plt.imshow(e_rho_real[:,:,0])
plt.colorbar()
plt.figure()
plt.imshow(e_rho_real[:,:,19])
plt.colorbar()

In [ ]:
### copy the 3PDF to have a better visualization 
rho_extended_x = np.concatenate((e_rho, e_rho), axis=0)
rho_extended_y = np.concatenate((rho_extended_x, rho_extended_x), axis=1)
rho_extended_z = np.concatenate((rho_extended_y, rho_extended_y), axis=2)
print(rho_extended_z.shape)

In [ ]:
### visualize the extended 3D PDF in the x-y plane
%matplotlib notebook
plt.style.use('acs-1col')
plt.imshow(np.rot90(rho_extended_z.real[:,:,0]), vmin=-2e8, vmax=2e8, extent=[-1, 1, -1, 1], cmap='seismic')
plt.title("z = 0")
plt.xlabel("x (l.u.)")
plt.ylabel("y (l.u.)")
plt.xlim(-1.03, 1.03)
plt.ylim(-1.03, 1.03)
cbar = plt.colorbar()
cbar.set_label('Patterson Function from \n Structure Factors')
plt.tight_layout()


In [ ]:
### visualize the extended 3D PDF in the x-z plane
%matplotlib notebook
print(rho_extended_z.real[:,0,:].shape)
plt.style.use('acs-1col')
plt.imshow(np.rot90(rho_extended_z.real[:,0,:]), vmin=-2e8, vmax=2e8, extent=[-1, 1, -1, 1], cmap='seismic')
plt.title("y = 0")
plt.xlabel("x (l.u.)")
plt.ylabel("z (l.u.)")
cbar = plt.colorbar()
cbar.set_label('Patterson Function from \n Structure Factors')
plt.tight_layout()



In [ ]:
### visualize the extended 3D PDF in the y-z plane
%matplotlib notebook
print(rho_extended_z.real[:,0,:].shape)
plt.style.use('acs-1col')
plt.imshow(np.rot90(rho_extended_z.real[0,:,:]), vmin=-2e8, vmax=2e8, extent=[-1, 1, -1, 1], cmap='seismic')
plt.title("x = 0")
plt.xlabel("y (l.u.)")
plt.ylabel("z (l.u.)")
cbar = plt.colorbar()
cbar.set_label('Patterson Function from \n Structure Factors')
plt.tight_layout()


